# 영화 관객 예측

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import warnings
warnings.filterwarnings('ignore')

In [2]:
train = pd.read_csv("C:/Users/run07/OneDrive/바탕 화면/영화 관객수/DATA/movies.csv")
test = train.drop('release_time',axis=1)

In [3]:
train.head(3)

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083


In [4]:
test.head(3)

,title,distributor,genre,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,123,15세 관람가,장철수,220775.25,4,343,4,6959083


배급사 전처리

In [5]:
train['distributor'] = train.distributor.str.replace("(주)", '')
test['distributor'] = train.distributor.str.replace("(주)", '')

In [6]:
def get_dis(x) :
    if 'CJ' in x or 'CGV' in x :
        return 'CJ'
    elif '쇼박스' in x :
        return '쇼박스'
    elif 'SK' in x :
        return 'SK'
    elif '리틀빅픽' in x :
        return '리틀빅픽처스'
    elif '스폰지' in x :
        return '스폰지'
    elif '싸이더스' in x :
        return '싸이더스'
    elif '에이원' in x :
        return '에이원'
    elif '마인스' in x :
        return '마인스'
    elif '마운틴픽' in x :
        return '마운틴픽처스'
    elif '디씨드' in x :
        return '디씨드'
    elif '드림팩트' in x :
        return '드림팩트'
    elif '메가박스' in x :
        return '메가박스'
    elif '마운틴' in x :
        return '마운틴'
    else :
        return x

In [7]:
train['distributor'] = train.distributor.apply(get_dis)
test['distributor'] = train.distributor.apply(get_dis)
# apply(함수,axis=0 or1)
# 첫번째 인자: 적용하고자 하는 함수 이름
# axis 인자: 함수를 행으로 적용할지 열로 적용할지 정해주는 인자
# 0:행, 1:열

장르별 영화 관객수 평균값으로 랭크 인코딩

In [8]:
train.groupby('genre').box_off_num.mean().sort_values()
# groupby: 주어진 데이터를 그룹 별로 구분하여 데이터를 보기 위해 사용되는 함수

genre
뮤지컬       6.627000e+03
다큐멘터리     6.717226e+04
서스펜스      8.261100e+04
애니메이션     1.819267e+05
멜로/로맨스    4.259680e+05
미스터리      5.275482e+05
공포        5.908325e+05
드라마       6.256898e+05
코미디       1.193914e+06
SF        1.788346e+06
액션        2.203974e+06
느와르       2.263695e+06
Name: box_off_num, dtype: float64

In [9]:
test.groupby('genre').time.mean().sort_values()

genre
애니메이션      82.571429
다큐멘터리      88.053763
멜로/로맨스     96.641026
공포        100.214286
SF        101.384615
미스터리      102.588235
드라마       104.651584
코미디       105.264151
뮤지컬       107.000000
서스펜스      112.000000
액션        114.464286
느와르       115.370370
Name: time, dtype: float64

배급사별 영화 관객수 중위값 기준으로 배급사 랭크 인코딩

In [10]:
train['genre_rank'] = train.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})
test['genre_rank'] = test.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})

In [11]:
tr_nm_rank = train.groupby('distributor').box_off_num.median().reset_index(name = 'num_rank').sort_values(by = 'num_rank')
tr_nm_rank

,distributor,num_rank
125,인피니티엔터테인먼트,2.0
51,고구마공작소,8.0
79,사람과 사람들,42.0
115,위드시네마,46.0
54,나우콘텐츠,54.0
...,...,...
128,전망좋은영화사,1214237.0
120,이십세기폭스코리아(),1422844.0
83,쇼박스,2138560.0
105,영구아트무비,2541603.0


In [12]:
te_nm_rank = test.groupby('distributor').box_off_num.median().reset_index(name = 'num_rank').sort_values(by = 'num_rank')

In [13]:
te_nm_rank

,distributor,num_rank
125,인피니티엔터테인먼트,2.0
51,고구마공작소,8.0
79,사람과 사람들,42.0
115,위드시네마,46.0
54,나우콘텐츠,54.0
...,...,...
128,전망좋은영화사,1214237.0
120,이십세기폭스코리아(),1422844.0
83,쇼박스,2138560.0
105,영구아트무비,2541603.0


In [14]:
te_nm_rank1 = test.groupby('distributor').time.median().reset_index(name = 'num_rank').sort_values(by = 'num_rank')

In [15]:
te_nm_rank1

,distributor,num_rank
135,크리에이티브컴즈(),61.0
149,한국YWCA연합회,61.0
124,인터콘미디어,63.0
73,머스트 씨 무비,67.0
84,스크린조이,67.0
...,...,...
59,두 엔터테인먼트,123.0
83,쇼박스,126.0
140,판다미디어,127.0
98,아방가르드 필름,127.0


In [16]:
tr_nm_rank['num_rank'] = [i + 1 for i in range(tr_nm_rank.shape[0])]

In [17]:
te_nm_rank['num_rank'] = [i + 1 for i in range(te_nm_rank.shape[0])]

In [18]:
tr_nm_rank

,distributor,num_rank
125,인피니티엔터테인먼트,1
51,고구마공작소,2
79,사람과 사람들,3
115,위드시네마,4
54,나우콘텐츠,5
...,...,...
128,전망좋은영화사,148
120,이십세기폭스코리아(),149
83,쇼박스,150
105,영구아트무비,151


In [19]:
te_nm_rank

,distributor,num_rank
125,인피니티엔터테인먼트,1
51,고구마공작소,2
79,사람과 사람들,3
115,위드시네마,4
54,나우콘텐츠,5
...,...,...
128,전망좋은영화사,148
120,이십세기폭스코리아(),149
83,쇼박스,150
105,영구아트무비,151


최종 데이터 병합

In [20]:
train = pd.merge(train, tr_nm_rank, how = 'left')

In [21]:
train

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,genre_rank,num_rank
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398,11,138
1,내부자들,쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501,12,150
2,은밀하게 위대하게,쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.25,4,343,4,6959083,11,150
3,나는 공무원이다,()NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.00,2,20,6,217866,9,145
4,불량남녀,쇼박스,코미디,2010-11-04,108,15세 관람가,신근호,1.00,1,251,2,483387,9,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,해무,()NEW,드라마,2014-08-13,111,청소년 관람불가,심성보,3833.00,1,510,7,1475091,8,145
596,파파로티,쇼박스,드라마,2013-03-14,127,15세 관람가,윤종찬,496061.00,1,286,6,1716438,8,150
597,살인의 강,마운틴픽처스,공포,2010-09-30,99,청소년 관람불가,김대현,NaN,0,123,4,2475,7,47
598,악의 연대기,CJ,느와르,2015-05-14,102,15세 관람가,백운학,NaN,0,431,4,2192525,12,146


In [22]:
test = pd.merge(test, te_nm_rank, how = 'left')

In [23]:
test

,title,distributor,genre,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,genre_rank,num_rank
0,개들의 전쟁,롯데엔터테인먼트,액션,96,청소년 관람불가,조병옥,NaN,0,91,2,23398,11,138
1,내부자들,쇼박스,느와르,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501,12,150
2,은밀하게 위대하게,쇼박스,액션,123,15세 관람가,장철수,220775.25,4,343,4,6959083,11,150
3,나는 공무원이다,()NEW,코미디,101,전체 관람가,구자홍,23894.00,2,20,6,217866,9,145
4,불량남녀,쇼박스,코미디,108,15세 관람가,신근호,1.00,1,251,2,483387,9,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,해무,()NEW,드라마,111,청소년 관람불가,심성보,3833.00,1,510,7,1475091,8,145
596,파파로티,쇼박스,드라마,127,15세 관람가,윤종찬,496061.00,1,286,6,1716438,8,150
597,살인의 강,마운틴픽처스,공포,99,청소년 관람불가,김대현,NaN,0,123,4,2475,7,47
598,악의 연대기,CJ,느와르,102,15세 관람가,백운학,NaN,0,431,4,2192525,12,146


In [24]:
test1 = pd.merge(test, te_nm_rank, how = 'right')

In [25]:
test1

,title,distributor,genre,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,genre_rank,num_rank
0,청아,인피니티엔터테인먼트,드라마,92,청소년 관람불가,김정호,NaN,0,3,4,2,8,1
1,타부 : 금지된 사랑,고구마공작소,멜로/로맨스,78,청소년 관람불가,고구마,17615314.0,1,27,3,8,5,2
2,90분,사람과 사람들,공포,94,청소년 관람불가,박선욱,NaN,0,55,2,42,7,3
3,나의 불행에는 이유가 있다,위드시네마,드라마,86,15세 관람가,한상민,NaN,0,0,4,46,8,4
4,러브멘토,나우콘텐츠,멜로/로맨스,74,청소년 관람불가,신혜성,1556.0,1,0,2,54,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,남자사용설명서,쇼박스,멜로/로맨스,116,15세 관람가,이원석,NaN,0,158,3,507913,5,150
596,상의원,쇼박스,드라마,127,15세 관람가,이원석,NaN,0,456,5,790370,8,150
597,파파로티,쇼박스,드라마,127,15세 관람가,윤종찬,496061.0,1,286,6,1716438,8,150
598,라스트 갓파더,영구아트무비,코미디,103,12세 관람가,심형래,7855474.0,1,31,5,2541603,9,151


In [26]:
test.fillna(0)

,title,distributor,genre,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num,genre_rank,num_rank
0,개들의 전쟁,롯데엔터테인먼트,액션,96,청소년 관람불가,조병옥,0.00,0,91,2,23398,11,138
1,내부자들,쇼박스,느와르,130,청소년 관람불가,우민호,1161602.50,2,387,3,7072501,12,150
2,은밀하게 위대하게,쇼박스,액션,123,15세 관람가,장철수,220775.25,4,343,4,6959083,11,150
3,나는 공무원이다,()NEW,코미디,101,전체 관람가,구자홍,23894.00,2,20,6,217866,9,145
4,불량남녀,쇼박스,코미디,108,15세 관람가,신근호,1.00,1,251,2,483387,9,150
...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,해무,()NEW,드라마,111,청소년 관람불가,심성보,3833.00,1,510,7,1475091,8,145
596,파파로티,쇼박스,드라마,127,15세 관람가,윤종찬,496061.00,1,286,6,1716438,8,150
597,살인의 강,마운틴픽처스,공포,99,청소년 관람불가,김대현,0.00,0,123,4,2475,7,47
598,악의 연대기,CJ,느와르,102,15세 관람가,백운학,0.00,0,431,4,2192525,12,146


모델링 데이터 구성
- 타겟 값인 관객수를 로그변환
- 상영등급은 더미 변수
- 출연 배우 수는 로그변환

In [27]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

In [28]:
X = train[['num_rank', 'time', 'num_staff', 'num_actor', 'genre_rank', 'screening_rat']]
y = np.log1p(train.box_off_num)

In [29]:
X

,num_rank,time,num_staff,num_actor,genre_rank,screening_rat
0,138,96,91,2,11,청소년 관람불가
1,150,130,387,3,12,청소년 관람불가
2,150,123,343,4,11,15세 관람가
3,145,101,20,6,9,전체 관람가
4,150,108,251,2,9,15세 관람가
...,...,...,...,...,...,...
595,145,111,510,7,8,청소년 관람불가
596,150,127,286,6,8,15세 관람가
597,47,99,123,4,7,청소년 관람불가
598,146,102,431,4,12,15세 관람가


In [30]:
y

0      10.060449
1      15.771725
2      15.755558
3      12.291640
4      13.088575
         ...    
595    14.204231
596    14.355762
597     7.814400
598    14.600565
599    15.784933
Name: box_off_num, Length: 600, dtype: float64

In [31]:
X = pd.get_dummies(columns = ['screening_rat'], data = X)
# 결측값을 제외하고 0과 1로 구성된 더미값이 만들어짐

In [32]:
X

,num_rank,time,num_staff,num_actor,genre_rank,screening_rat_12세 관람가,screening_rat_15세 관람가,screening_rat_전체 관람가,screening_rat_청소년 관람불가
0,138,96,91,2,11,0,0,0,1
1,150,130,387,3,12,0,0,0,1
2,150,123,343,4,11,0,1,0,0
3,145,101,20,6,9,0,0,1,0
4,150,108,251,2,9,0,1,0,0
...,...,...,...,...,...,...,...,...,...
595,145,111,510,7,8,0,0,0,1
596,150,127,286,6,8,0,1,0,0
597,47,99,123,4,7,0,0,0,1
598,146,102,431,4,12,0,1,0,0


In [33]:
X['num_actor'] = np.log1p(X['num_actor'])

In [34]:
X['num_actor']

0      1.098612
1      1.386294
2      1.609438
3      1.945910
4      1.098612
         ...   
595    2.079442
596    1.945910
597    1.609438
598    1.609438
599    1.791759
Name: num_actor, Length: 600, dtype: float64

In [35]:
target = test[['num_rank', 'time', 'num_staff', 'num_actor', 'genre_rank', 'screening_rat']]

In [36]:
target

,num_rank,time,num_staff,num_actor,genre_rank,screening_rat
0,138,96,91,2,11,청소년 관람불가
1,150,130,387,3,12,청소년 관람불가
2,150,123,343,4,11,15세 관람가
3,145,101,20,6,9,전체 관람가
4,150,108,251,2,9,15세 관람가
...,...,...,...,...,...,...
595,145,111,510,7,8,청소년 관람불가
596,150,127,286,6,8,15세 관람가
597,47,99,123,4,7,청소년 관람불가
598,146,102,431,4,12,15세 관람가


In [37]:
target = pd.get_dummies(columns = ['screening_rat'], data = target)

In [38]:
target

,num_rank,time,num_staff,num_actor,genre_rank,screening_rat_12세 관람가,screening_rat_15세 관람가,screening_rat_전체 관람가,screening_rat_청소년 관람불가
0,138,96,91,2,11,0,0,0,1
1,150,130,387,3,12,0,0,0,1
2,150,123,343,4,11,0,1,0,0
3,145,101,20,6,9,0,0,1,0
4,150,108,251,2,9,0,1,0,0
...,...,...,...,...,...,...,...,...,...
595,145,111,510,7,8,0,0,0,1
596,150,127,286,6,8,0,1,0,0
597,47,99,123,4,7,0,0,0,1
598,146,102,431,4,12,0,1,0,0


In [39]:
target['num_actor'] = np.log1p(target['num_actor'])

In [40]:
target['num_actor']

0      1.098612
1      1.386294
2      1.609438
3      1.945910
4      1.098612
         ...   
595    2.079442
596    1.945910
597    1.609438
598    1.609438
599    1.791759
Name: num_actor, Length: 600, dtype: float64

10Fold로 교차검증

In [41]:
kf = KFold(n_splits =  10, shuffle = True, random_state = 42)

GradientBoostingRegressor 모델 사용

In [42]:
gbm = GradientBoostingRegressor(random_state = 42)

In [43]:
rmse_list = []
gb_pred = np.zeros((test.shape[0]))
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    gbm.fit(tr_x, tr_y) # 입력된 데이터의 형태에 맞춰서 데이터를 변환하기 위한 사전 구조 작업
    
    pred = np.expm1([0 if x < 0 else x for x in gbm.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in gbm.predict(target)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))
    # sqrt(x) 메서드는 숫자 x의 제곱근을 반환
    # mean_squared_error(): 평균 제곱 오차
    
    rmse_list.append(rmse)
    
    gb_pred += (sub_pred / 10)
    

In [44]:
np.mean(rmse_list)

1136799.2172945936

In [45]:
gb_pred 

array([3.13227194e+04, 3.74640950e+06, 4.49153279e+06, 1.17896768e+05,
       7.72230075e+05, 4.12229390e+05, 1.82846289e+04, 5.69958670e+05,
       4.77743775e+03, 9.93080505e+02, 6.25427691e+02, 1.34669171e+03,
       1.40938521e+03, 1.43936670e+04, 6.19367985e+04, 1.51139700e+04,
       6.73825294e+06, 4.08046685e+04, 3.47252537e+05, 8.08843617e+03,
       1.21182243e+06, 6.55966826e+04, 3.52774267e+03, 4.40500043e+05,
       2.98386121e+03, 9.02835840e+03, 3.20956069e+03, 6.25427691e+02,
       1.98678824e+05, 1.17530676e+03, 5.18940624e+05, 4.10955657e+02,
       9.61304528e+04, 1.45017861e+03, 1.23734928e+06, 4.62913601e+03,
       1.87474685e+03, 9.22790909e+03, 3.77263690e+03, 1.30541625e+06,
       4.42330564e+04, 4.59349353e+04, 7.12110067e+02, 2.07080986e+03,
       1.79016291e+06, 1.02698616e+03, 1.88307934e+05, 5.63842123e+05,
       2.13021793e+06, 2.69373131e+04, 4.92629036e+02, 4.34380345e+05,
       1.63852541e+04, 3.76347944e+04, 5.32805642e+02, 1.81352285e+06,
      

In [46]:
gb_pred.shape

(600,)